# Inversion example of diffusion + tracer mineral (uranium salt) dissolution inversion in 2d

- Importation of the required modules

In [ ]:
import logging
from pathlib import Path
from typing import Tuple, Sequence
import os
import copy
import tempfile
from enum import Enum
import pyrtid
from pyrtid.utils.wellfield import gen_wells_coordinates
import pyrtid.forward as dmfwd
import pyrtid.inverse as dminv
import pyrtid.utils.spde as spde
from pyrtid.utils import indices_to_node_number, NDArrayFloat
from sksparse.cholmod import cholesky
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LogNorm
from matplotlib.animation import HTMLWriter
from IPython.display import HTML
import nested_grid_plotter as ngp
import numpy as np
import pandas as pd
import gstools as gs

- Set-up logging level

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("this is a logging test")

- Check package/software versions


In [ ]:
pyrtid.utils.show_versions()

- Random number generator with a given seed for reproducible results

In [ ]:
rng = np.random.default_rng(2023)

- Create a directory to store the exported figures

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
fig_save_path = Path(ipynb_path, "exported_figures")
fig_save_path.mkdir(parents=True, exist_ok=True)  # make sure that the directory exists

- Define some configurations for the plots

In [ ]:
# Some configs for the plots
new_rc_params = {
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica", "DejaVu Sans"],
    "font.size": 16,
    "text.usetex": False,
    "savefig.format": "svg",
    "svg.fonttype": "none",  # to store text as text, not as path
    "savefig.facecolor": "w",
    "savefig.edgecolor": "k",
    "savefig.dpi": 300,
    "figure.constrained_layout.use": True,
    "figure.facecolor": "w",
    # "axes.facecolor": "w",
}
csfont = {"fontname": "Comic Sans MS"}
hfont = {"fontname": "Helvetica"}
plt.plot()
plt.close()  # required for the plot to update
plt.rcParams.update(new_rc_params)

## Forward problem

- Define a very simple pure diffusion case in 1D.

In [ ]:
# Grid
nx = 57  # number of voxels along the x axis
ny = 57  # number of voxels along the y axis
nz = 1
dx = 5.0  # voxel dimension along the x axis
dy = 5.0  # voxel dimension along the y axis
dz = 1.0

# Timesteps
duration_in_days = 200
dt = 3600 * 6  # timestep in seconds
nt = int(duration_in_days * 3600 * 24 / dt)  # number of time steps

# Hydro parameters
flow_regime = "transient"
diffusion_coef = 1e-6  # general initial diffusion coefficient [m2/s]
permeability = 1e-4  # general permeability
porosity = 0.23  # general porosity [fraction]
storage_coefficient = 1e-3

# Chemistry parameters
c0 = 1.0e-10  # general initial concentration [molal]
c_inj = 1e-10  # molal

M0 = 0.001  # mineral grade [mol/kg] -> kg of water
kv = -6.9e-9  # kinetic rate,       [mol/m2/s]
moleweight = 270.0  # molar weight [g/mol]
surface = 500  # cm2/g
As = moleweight * surface / 1e4  # specific area [m2/mol]
logK = 3.2
Ks = 1.0 / pow(10, logK)  # solubility constant [no unit]

- Creation of a network of injectors and producers in shape of hexagons.

In [ ]:
# locations in the grid
# selection = index of cells to select. Cells are sorted by y, and if that's equal by x (increasing order).
cell_radius = 40.0
inj_grid_locations, prod_grid_locations, polygons = gen_wells_coordinates(
    -25.0,
    65.0,
    175.0,
    235.0,
    radius=cell_radius,
    rotation=-30,
    selection=[0, 1, 2, 4, 5, 6, 7],
)

In [ ]:
plotter_wf = ngp.NestedGridPlotter(fig_params={"figsize": (5, 5)})

for i, j in inj_grid_locations:
    plotter_wf.ax_dict["ax1-1"].plot(i, j, "ko")

for i, j in prod_grid_locations:
    plotter_wf.ax_dict["ax1-1"].plot(i, j, "ro")

patches = []
for polygon in polygons:
    # Add the polygon to the collection of patches
    xy = np.array(polygon)
    patches.append(Polygon(xy, closed=True, facecolor=None))
    # Plot the number of the polygon
p = PatchCollection(patches, alpha=0.75)
p.set_facecolors("white")
p.set_edgecolors("darkgrey")
p.set_linewidth(0.8)
p.set_linestyle("-")
plotter_wf.ax_dict["ax1-1"].add_collection(p)

for i, prod_coords in enumerate(prod_grid_locations):
    plotter_wf.ax_dict["ax1-1"].text(prod_coords[0] - 2, prod_coords[1] + 8, i)

plotter_wf.ax_dict["ax1-1"].set_aspect("equal", adjustable="box")
plotter_wf.ax_dict["ax1-1"].set_xlim(0, nx * dx)
plotter_wf.ax_dict["ax1-1"].set_ylim(0, ny * dy)
plotter_wf.ax_dict["ax1-1"].set_xlabel("X (m)", fontweight="bold")
plotter_wf.ax_dict["ax1-1"].set_ylabel("Y (m)", fontweight="bold")

fname = "7_cells_wellfield"
for format in ["png", "pdf"]:
    plotter_wf.fig.savefig(
        str(fig_save_path.joinpath(f"{fname}.{format}")), format=format
    )

- Create the flowrates for the wells: 12 m3/h decreasing following an exponential... for the producers with a balanced injection (2 m3/h per associated cell for the injectors).

In [ ]:
def gen_flowrates(amplitude: float, coef: float, nt: int, dt: float) -> NDArrayFloat:
    """Generate flowrates with a given amplitude and decrease coefficient.

    Parameters
    ----------
    amplitude : float
        Amplitude in m3/h.
    coef : float
        Decrease coefficient.
    nt : int
        Number of timesteps
    dt : float
        Timesteps.

    Returns
    -------
    np.ndarray
        The flowrates.
    """
    return amplitude * np.exp(coef * np.arange(nt) * dt)

- Generate a flowrates with an initial amplitude at 1 m3/h and plot it
- For now steady state

In [ ]:
init_prod_flowrates = 8.0  # m3/s
flowrates = init_prod_flowrates * gen_flowrates(
    1.0, -0.0015, duration_in_days + 1, dt=1.0
)  # m3/h
time_in_days = np.arange(duration_in_days + 1)
# plt.plot(np.arange(nt + 1) * dt / 3600 / 24, flowrates)
# plt.ylim(0.0, init_prod_flowrates * 1.1)

plt.plot(np.arange(duration_in_days + 1), flowrates)
plt.ylim(0.0, init_prod_flowrates * 1.1)
plt.ylabel("flowrates [m3/h]", fontweight="bold")
plt.xlabel("time [d]", fontweight="bold")

fname = "flowrates_decrease"
for format in ["png", "pdf"]:
    plt.savefig(str(fig_save_path.joinpath(f"{fname}.{format}")), format=format)

### Mineral grade: reference vs estimated

The mineral grades are defined in [mol/kg]. We calculate the conversion factor to obtain ppm and perform easier mass balances.

ConvU: parameter for converting the mineral content in [mol/kg] to metal grade in [ppm]. Note that the ConvU parameter is specific to the Uranium carrier phase: here Uraninite.

$C_{Uraninite}[\frac{mol}{kg}] = convU \times T_{Uranium}[ppm]$


and


$conv_u = \frac{1.023 \times density_{rock}}{238*porosity_{rock} \times 1000}$

Note the 1.023 is the conversion factor from molar mass to molal mass in CHESS.

In [ ]:
rock_density = 1.63
conv_u_grade: NDArrayFloat = np.array(
    [1.023 * rock_density / (238.0 * porosity * 1000)]
)

- Conversion factor for uranium from mol/l to mg/l

In [ ]:
conv_u_conc: float = (270.03 - 0.0016 * 2) * 1e3

- Using the SPDE approach, generate a random field

In [ ]:
len_scale = 20  # m
kappa = 1 / len_scale
alpha = 1
# min_val = 32 * conv_u_grade  # 32 ppm
# max_val = 300 * conv_u_grade  # 700 ppm
# # Compute the mean and the standard deviation that the distribution should have so that
# # <99% of the values are between min and max ~ 6 sigmas
# mean = (max_val + min_val) / 2
# stdev = (max_val - min_val) / 2 / 3  # std ~ 1/6 of the distribution interva

mean = 300  # trend of the field
std = 150  # standard deviation of the field

# Create a precison matrix
Q_ref = spde.get_precision_matrix(nx, ny, dx, dy, kappa, alpha, spatial_dim=2)
cholQ_ref = cholesky(Q_ref)
# Non conditonal simulation -> change the random state to obtain a different field
simu_ = spde.simu_nc(cholQ_ref, random_state=2023).reshape(ny, nx).T
reference_grade = np.abs(simu_ * std + mean)

In [ ]:
plt.imshow(
    reference_grade.T,
    origin="lower",
    cmap=plt.get_cmap("jet"),
    aspect="equal",
    vmin=50.0,
    vmax=650,
)
plt.colorbar()

- Select some points from this field (drilling places) -> green dots on the map

In [ ]:
_ix = np.array([int(nx / 4), 2 * int(nx / 4), 3 * int(nx / 4)])
_iy = np.array([int(ny / 5), 2 * int(ny / 5), 3 * int(ny / 5), 4 * int(ny / 5)])

dat_coords = np.array(np.meshgrid(_ix, _iy)).reshape(2, -1)
# Get the node numbers
dat_nn = indices_to_node_number(dat_coords[0, :], nx, dat_coords[1, :])
dat_val = reference_grade.ravel("F")[dat_nn]

for i, j in dat_coords.T:
    plotter_wf.ax_dict["ax1-1"].plot((i + 0.5) * dx, (j + 0.5) * dy, "go")

plotter_wf.fig

In [ ]:
kappa_est = 25
alpha = 2
# Create a precison matrix
Q_init = spde.get_precision_matrix(nx, ny, dx, dy, kappa, alpha, spatial_dim=2)
# Decompose with cholesky
cholQ_init = cholesky(Q_init)

- Generate 3 conditional simulations with more or less error on the known data with a varigram which is not exactly correct + one constant field (witness case)

In [ ]:
estimate_grade_flat = np.ones((nx, ny)) * 200  # 200 ppm

# Condition with the exact data
dat_var_ref = np.ones(dat_val.size) * 0.01**2
Q_init_c = spde.condition_precision_matrix(Q_init, dat_nn, dat_var_ref)
cholQ_init_c = cholesky(Q_init_c)
# estimate_grade_krig2 = spde.kriging(Q_init, dat_val, dat_nn).reshape(ny, nx).T

# Compute the average on the data points (trend)
av_dat = np.average(dat_val)
# Remove the trend from the residuals
dat_4_sim2 = dat_val - av_dat

estimate_grade_simu = np.abs(
    spde.simu_c(
        cholQ_init,
        Q_init_c,
        cholQ_init_c,
        dat_4_sim2,
        dat_nn,
        dat_var_ref,
        random_state=2028,
    )
    .reshape(ny, nx)
    .T
    + av_dat
)

# Generate new points with error -> systematic bias 200 ppm + some variance on the mesures
dat_val_wrong = (dat_val - 200) + 100 * np.random.default_rng(2023).normal(
    scale=1.0, size=dat_val.size
)
# Compute the average on the data points (trend)

av_dat_wrong = np.average(dat_val_wrong)
# Remove the trend from the residuals
dat_4_sim2 = dat_val_wrong - av_dat_wrong

estimate_grade_simu2 = np.abs(
    spde.simu_c(
        cholQ_init,
        Q_init_c,
        cholQ_init_c,
        dat_4_sim2,
        dat_nn,
        dat_var_ref,
        random_state=2028,
    )
    .reshape(ny, nx)
    .T
    + av_dat_wrong
)
# estimate_grade_krig2 = spde.kriging(Q_init, dat_val, dat_nn).reshape(ny, nx).T
# estimate_grade_krig2 = spde.kriging(Q_init, dat_val, dat_nn).reshape(ny, nx).T

In [ ]:
plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (14, 3.5)},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "axes1-4"]], sharey=True, sharex=True
        )
    },
)

ngp.multi_imshow(
    axes=plotter.axes,
    fig=plotter.fig,
    data={
        "Reference": reference_grade,
        "Cst estimate": estimate_grade_flat,
        "Cond sim dat ok": estimate_grade_simu,
        "Cond sim dat bad": estimate_grade_simu2,
    },
    cbar_title="Field value",
    imshow_kwargs={
        "cmap": plt.get_cmap("jet"),
        "extent": [0.0, nx * dx, 0.0, ny * dy],
        "vmin": 50,
        "vmax": 650,
    },
    xlabel="X (m)",
    ylabel="Y (m)",
)

- Plot the reserves per mesh

In [ ]:
def get_cell_reserves(
    u_field_grades: NDArrayFloat,
    cell_polygons: Sequence[Sequence[Tuple[float, float]]],
    dx: float,
    dy: float,
    dz: float,
) -> Sequence[float]:
    """Return the reserves in t under the given technological cell.

    Parameters
    ----------
    u_field_grades : NDArrayFloat
        Uraninite grades field in ppm. 2D Array.
    dx : float
        X dimension of one mesh (m).
    dy : float
        Y dimension of one mesh (m).
    dz : float
        Z dimension of one mesh (m).
    mask: NDArrayFloat
        Mask to apply before doing the selection.
    Returns
    -------
    float
        The associated uranium reserves in t.
    """
    # flatten points coordinates
    _x, _y, _z = hytec_node_number_to_indices(np.arange(nx * ny), nx, ny)
    _x = _x * dx + dx / 2
    _y = _y * dy + dy / 2
    grid_coords = np.array((_x, _y)).T

    cell_volume = dx * dy * dz  # m3
    # Conversion factor to go from a number of ppm per unit of volume to a mass in t (all cells are the same)
    conv_factor = cell_volume * rock_density * 1e-6

    reserves_list = []

    mask_sum = None
    for cell_polygon in cell_polygons:
        # Select the mesh that belongs to the polygon
        path = mpl.path.Path(cell_polygon)
        mask = path.contains_points(grid_coords)
        if mask_sum is not None:
            mask = np.logical_and(mask, ~mask_sum)
            mask_sum = np.logical_or(mask, mask_sum)
        else:
            mask_sum = mask
        reserves_list.append(float(u_field_grades[mask].sum() * conv_factor))
    return reserves_list

In [ ]:
# Compute the reserve associated to the cells in t
# Need to transpose for the display
reference_reserves = get_cell_reserves(reference_grade.T.ravel(), polygons, dx, dy, dz)
estimated_reserves_flat = get_cell_reserves(
    estimate_grade_flat.T.ravel(), polygons, dx, dy, dz
)
estimated_reserves_simu_1 = get_cell_reserves(
    estimate_grade_simu.T.ravel(), polygons, dx, dy, dz
)
estimated_reserves_simu_2 = get_cell_reserves(
    estimate_grade_simu2.T.ravel(), polygons, dx, dy, dz
)

plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 5)},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "ax1-4"]], sharey=True, sharex=True
        )
    },
)

for ax, reserves in zip(
    plotter.ax_dict.values(),
    (
        reference_reserves,
        estimated_reserves_flat,
        estimated_reserves_simu_1,
        estimated_reserves_simu_2,
    ),
):
    for i, j in inj_grid_locations:
        ax.plot(i, j, "ko")

    for i, j in prod_grid_locations:
        ax.plot(i, j, "ro")

    patches = []
    for i, cell_polygon in enumerate(polygons):
        # Add the polygon to the collection of patches
        xy = np.array(cell_polygon)
        patches.append(Polygon(xy, closed=True, facecolor=None))

        centroid = (xy.mean(axis=0)[0] - 3, xy.mean(axis=0)[1] + 8)

        ax.text(*centroid, f"{reserves[i]:.2f} t", fontsize=10, fontweight="bold")

        # Plot the number of the polygon
    p = PatchCollection(patches, alpha=0.5, cmap=plt.get_cmap("jet"))
    # p.set_facecolors("white")
    p.set_edgecolors("black")
    p.set_linewidth(0.6)
    p.set_linestyle("-")
    p.set_array(np.array(reserves, dtype=float))
    # p.set_label(np.array(np.round(reserves, 2), dtype=float))
    p.set_clim(1, 5.5)

    ax.add_collection(p)

# Add a common colorbar
plotter.fig.colorbar(p, label="Uranium [t]")

# Box dimensions
for ax in plotter.ax_dict.values():
    ax.set_xlim(0, nx * dx)
    ax.set_ylim(0, ny * dy)
    ax.set_aspect("equal", adjustable="box")

plotter.subfigs["fig0"].suptitle(
    "Metal per production cell [t]", fontweight="bold", fontsize=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.ax_dict["ax1-1"].set_title('Reference ("True")', fontweight="bold")
plotter.ax_dict["ax1-2"].set_title("Initial guess (flat)", fontweight="bold")
plotter.ax_dict["ax1-2"].set_title("Initial guess (simu 1)", fontweight="bold")
plotter.ax_dict["ax1-2"].set_title("Initial guess (simu 2)", fontweight="bold")

fname = "reserves_per_cells_true_vs_estimated"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

In [ ]:
initial_u_dev_flat = np.sum(estimated_reserves_flat) - np.sum(reference_reserves)
initial_u_dev_simu_1 = np.sum(estimated_reserves_simu_1) - np.sum(reference_reserves)
initial_u_dev_simu_2 = np.sum(estimated_reserves_simu_2) - np.sum(reference_reserves)

initial_u_dev_flat_frac = initial_u_dev_flat / np.sum(reference_reserves)
initial_u_dev_simu_1_frac = initial_u_dev_simu_1 / np.sum(reference_reserves)
initial_u_dev_simu_2_frac = initial_u_dev_simu_2 / np.sum(reference_reserves)

logging.info(
    f"initial_U_dev_flat = {initial_u_dev_flat:.2f} t ({initial_u_dev_flat_frac * 100:.2f}%)"
)
logging.info(
    f"initial_U_dev_simu_1 = {initial_u_dev_simu_1:.2f} t ({initial_u_dev_simu_1_frac * 100:.2f}%)"
)
logging.info(
    f"initial_U_dev_simu_2 = {initial_u_dev_simu_2:.2f} t ({initial_u_dev_simu_2_frac * 100:.2f}%)"
)

## Forward problem in demonstrator

In [ ]:
time_params = dmfwd.TimeParameters(
    duration=duration_in_s, dt_init=dt_init, dt_max=dt_max, dt_min=dt_min
)
geometry = dmfwd.Geometry(nx=nx, ny=ny, dx=dx, dy=dx)
fl_params = dmfwd.FlowParameters(
    permeability=permeability,
    storage_coefficient=storage_coefficient,
    regime=dmfwd.FlowRegime.TRANSIENT,
    is_numerical_acceleration=False,
    tolerance=1e-12,
)
tr_params = dmfwd.TransportParameters(
    diffusion=diffusion_coef,
    porosity=porosity,
    crank_nicolson_diffusion=1.0,
    crank_nicolson_advection=0.5,
    is_numerical_acceleration=False,
    tolerance=1e-15,
)
gch_params = dmfwd.GeochemicalParameters(conc=c0, grade=M0, kv=kv, As=As, Ks=Ks)

base_model = dmfwd.ForwardModel(geometry, time_params, fl_params, tr_params, gch_params)

- Add boundary conditions

In [ ]:
# The fact to place boundary conditions causes weird behavior

# base_model.add_boundary_conditions(
#     dmfwd.ConstantHead(span=(slice(0, 1), slice(None)))
# )
# base_model.add_boundary_conditions(
#     dmfwd.ConstantHead(span=(slice(None), slice(0, 1)))
# )
# base_model.add_boundary_conditions(
#     dmfwd.ConstantHead(span=(slice(nx - 1, nx), slice(None)))
# )
# base_model.add_boundary_conditions(
#     dmfwd.ConstantHead(span=(slice(None), slice(ny - 1, ny)))
# )
base_model.fl_model.head[0, :, :] = 0.0
base_model.fl_model.head[-1, :, :] = 0.0
base_model.fl_model.head[:, 0, :] = 0.0
base_model.fl_model.head[:, -1, :] = 0.0

- Add source and sink terms

In [ ]:
def flw_to_new_dt(
    flw: NDArrayFloat, times: NDArrayFloat, dt: float
) -> Tuple[NDArrayFloat, NDArrayFloat]:
    duration_in_days = flw.size - 1
    nt = int(duration_in_days * 24 * 3600 / dt)

    # Time in second, for each timestep
    new_times = np.arange(nt + 1) * dt

    # Flowrates
    new_flw = np.zeros(nt + 1)

    index = 0
    for new_index in range(nt + 1):
        time = new_index * dt  # in second
        while time > times[index] * 3600 * 24:
            index += 1
        new_flw[new_index] = flw[index]
    return new_times, new_flw


_times, _flowrates = flw_to_new_dt(flowrates, time_in_days, dt)
_times.size
_flowrates.size

In [ ]:
prod_flw = -_flowrates / 3600  # m3/h to m3/s
# times = np.arange(stop=nt + 1, start=0) * dt

for count_prod, (x, y) in enumerate(prod_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    name = f"producer_{ix}_{iy}"

    sink_term = dmfwd.SourceTerm(
        name,
        node_ids=np.array([indices_to_hytec_node_number(ix, nx, iy)]),
        times=_times,
        flowrates=prod_flw,
        concentrations=np.zeros(_times.shape),
    )
    base_model.add_src_term(sink_term)


for count_inj, (x, y) in enumerate(inj_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    name = f"injector_{ix}_{iy}"

    nb_prod_well_linked = 0
    # Find the number of producers that the injector is linked with
    for xp, yp in prod_grid_locations:
        # we take 20% margin on the cell radius
        if np.sqrt((x - xp) ** 2 + (y - yp) ** 2) < cell_radius * 1.2:
            nb_prod_well_linked += 1

    source_term = dmfwd.SourceTerm(
        name,
        node_ids=np.array([indices_to_hytec_node_number(ix, nx, iy)]),
        times=_times,
        flowrates=-prod_flw
        / 6.0
        * nb_prod_well_linked,  # /6.0 because of hexagonal cells
        concentrations=np.ones(_times.shape) * c_inj,  # injection concentration
    )
    base_model.add_src_term(source_term)

- Check the hydraulic balance

In [ ]:
total_prod_flow = 0
total_inj_flow = 0

for source_term in base_model.source_terms:
    _dt = np.diff(_times)  # timesteps in s
    total_inj_flow += np.sum(
        np.where(source_term.flowrates > 0, source_term.flowrates, 0.0)[1:] * _dt
    )
    total_prod_flow += np.sum(
        np.where(source_term.flowrates < 0, -source_term.flowrates, 0.0)[1:] * _dt
    )

print(f"total_prod_flow = {total_prod_flow} m3")
print(f"total_inj_flow = {total_inj_flow} m3")

# This test should fail if there is any issue !
assert np.round(total_prod_flow, 4) == np.round(total_inj_flow, 4)

In [ ]:
source_term.times.size

- Create two models

In [ ]:
model_reference = copy.deepcopy(base_model)
model_estimate_flat = copy.deepcopy(base_model)
model_estimate_simu_1 = copy.deepcopy(base_model)
model_estimate_simu_2 = copy.deepcopy(base_model)

# Exact initial uranium grade
model_reference.tr_model.set_initial_grade(reference_grade * conv_u_grade)
# Estimated initial uranim grade
model_estimate_flat.tr_model.set_initial_grade(estimate_grade_flat * conv_u_grade)
model_estimate_simu_1.tr_model.set_initial_grade(estimate_grade_simu * conv_u_grade)
model_estimate_simu_2.tr_model.set_initial_grade(estimate_grade_simu2 * conv_u_grade)

- Run the models

In [ ]:
solver_reference = dmfwd.ForwardSolver(model_reference)
solver_reference.solve()

In [ ]:
# solver_estimate_flat = dmfwd.ForwardSolver(model_estimate_flat)
# solver_estimate_flat.solve()

In [ ]:
# solver_estimate_simu_1 = dmfwd.ForwardSolver(model_estimate_simu_1)
# solver_estimate_simu_1.solve()

In [ ]:
# solver_estimate_simu_2 = dmfwd.ForwardSolver(model_estimate_simu_2)
# solver_estimate_simu_2.solve()

In [ ]:
time_index = 3

plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (6, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1"]], sharey=True, sharex=True),
    },
)

ngp.multi_imshow(
    axes=plotter.axes[:1],
    fig=plotter.fig,
    data={
        "Head [m]": model_reference.fl_model.head[:, :, time_index],
    },
    cbar_title="Head $[m]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy], "aspect": "equal"},
)

Y, X = np.meshgrid((np.arange(nx) + 0.5) * dx, (np.arange(ny) + 0.5) * dy)
plotter.ax_dict["ax1-1"].quiver(
    X,
    Y,
    model_reference.fl_model.u_darcy_x_center[:, :, time_index],
    model_reference.fl_model.u_darcy_y_center[:, :, time_index],
    color="C0",
    scale_units="xy",
)

# plotter.ax_dict["ax1-1"].set_title("Heads [m]", fontweight="bold")
# plotter.ax_dict["ax1-2"].set_title("Darcy velocity [m/s]", fontweight="bold")

plotter.subfigs["fig0"].suptitle(
    "Heads and Darcy velocities", fontweight="bold", fontsize=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

fname = "heads_and_velocities"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Plot the concentration evolution

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (16, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "ax1-4"]], sharey=True, sharex=True
        ),
    },
)
nb_frames = min(15, model_reference.time_params.nt)

plotter.animated_multi_imshow(
    ax_names=list(plotter.ax_dict.keys()),
    fig=plotter.fig,
    data={
        "Reference": model_reference.tr_model.conc[:, :, :] * conv_u_conc,
        "Estimate (flat)": model_estimate_flat.tr_model.conc[:, :, :] * conv_u_conc,
        "Estimate (simu_1)": model_estimate_simu_1.tr_model.conc[:, :, :] * conv_u_conc,
        "Estimate (simu_2)": model_estimate_simu_2.tr_model.conc[:, :, :] * conv_u_conc,
    },
    cbar_title="Concentration $[mg/l]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy], "aspect": "equal"},
    nb_frames=nb_frames,
    # is_add_grid=True,
)

# Add animated text
plotter.plot_animated_text(
    plotter.ax_dict["ax1-1"],
    x=5.0,
    y=290.0,
    s=[f"{d:.2f} d" for d in np.arange(nb_frames) * duration_in_days / (nb_frames - 1)],
    fontweight="bold",
)

plotter.subfigs["fig0"].suptitle(
    "Concentration evolution (PyRTID)", fontweight="bold", size=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("fwd_conc_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(str(fname_html), writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

In [ ]:
# Plot a grade evolution animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (16, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "ax1-4"]], sharey=True, sharex=True
        ),
    },
)
nb_frames = nb_frames = min(15, model_reference.time_params.nt)

plotter.animated_multi_imshow(
    ax_names=list(plotter.ax_dict.keys()),
    fig=plotter.fig,
    data={
        "Reference": model_reference.tr_model.grade[:, :, :] / conv_u_grade,
        "Estimate (flat)": model_estimate_flat.tr_model.grade[:, :, :] / conv_u_grade,
        "Estimate (simu_1)": model_estimate_simu_1.tr_model.grade[:, :, :]
        / conv_u_grade,
        "Estimate (simu_2)": model_estimate_simu_2.tr_model.grade[:, :, :]
        / conv_u_grade,
    },
    cbar_title="Concentration $[mg/l]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy], "aspect": "equal"},
    nb_frames=nb_frames,
    # is_add_grid=True,
)

# Add animated text
plotter.plot_animated_text(
    plotter.ax_dict["ax1-1"],
    x=5.0,
    y=290.0,
    s=[f"{d:.2f} d" for d in np.arange(nb_frames) * duration_in_days / (nb_frames - 1)],
    fontweight="bold",
)

plotter.subfigs["fig0"].suptitle("Grade evolution (PyRTID)", fontweight="bold", size=16)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("fwd_conc_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(str(fname_html), writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Plot the concentrations at the production wells

In [ ]:
# Indicate the order in which to plot
obs_plot_locations = ["ax2-3", "ax1-2", "ax3-2", "ax2-2", "ax1-1", "ax3-1", "ax2-1"]

plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 8)},
    subfigs_params={
        "ncols": 3  # np.unique(np.array(prod_grid_locations)[:, 0].round()).size,
    },
    subplots_mosaic_params={
        "left_col": dict(
            mosaic=[["ghost1"], ["ax1-1"], ["ax1-2"], ["ghost2"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
        "centered_col": dict(
            mosaic=[["ax2-1"], ["ax2-2"], ["ax2-3"]], sharey=True, sharex=True
        ),
        "right_col": dict(
            mosaic=[["ghost3"], ["ax3-1"], ["ax3-2"], ["ghost4"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
    },
)

# hide axes and borders
plotter.ax_dict["ghost1"].axis("off")
plotter.ax_dict["ghost2"].axis("off")
plotter.ax_dict["ghost3"].axis("off")
plotter.ax_dict["ghost4"].axis("off")

# Static plot
for count, ((x, y), ax_name) in enumerate(zip(prod_grid_locations, obs_plot_locations)):
    ix = int(x / dx - 0.5)
    iy = int(y / dy - 0.5)
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ x={x:.1f}m, y={y:.1f}m"

    ax = plotter.ax_dict[ax_name]
    ax.set_title(obs_well_name, fontweight="bold", fontsize=12)

    data = {
        "Reference.": {
            "data": model_reference.tr_model.conc[ix, iy, :],
            "kwargs": {"c": "b"},
        },
        "A priori cst.": {
            "data": model_estimate_flat.tr_model.conc[ix, iy, :],
            "kwargs": {"c": "r"},
        },
        "A priori simu_1.": {
            "data": model_estimate_simu_1.tr_model.conc[ix, iy, :],
            "kwargs": {"c": "g", "linestyle": "--"},
        },
        "A priori simu_2C": {
            "data": model_estimate_simu_2.tr_model.conc[ix, iy, :],
            "kwargs": {"c": "k", "linestyle": "--"},
        },
    }

    for k, v in data.items():
        ax.plot(v["data"], **v["kwargs"], label=k)

    ax.set_xlabel("Time", fontweight="bold")
    ax.set_xlabel("T_Cinet [mol/s]", fontweight="bold")

plotter.add_fig_legend(ncol=2)
plotter.fig.supxlabel("Time [d]", fontweight="bold")
plotter.fig.supylabel("T_Cinet [mol/s]", fontweight="bold")

fname = "T_Cinet_true_at_producers"
for format in ["png", "pdf"]:
    plotter.savefig(str(fig_save_path.joinpath(f"{fname}.{format}")), format=format)

## Forward problem in HYTEC

- Create an empty simulation in a non existing folder

In [ ]:
simu_base = HytecSimulation("simu_base", Path.cwd().joinpath("simu_base"))

- Add a TDB file

In [ ]:
simu_base.link_tdb("./../../../../../TDB/chess.tdb")  # This is relative to the htc file

- Define the output format

In [ ]:
simu_base.model.add_sub_item(hymd.OutputFormat("vtk"))

- Hydrodynamic model and its parameters definition


In [ ]:
hmodel = hymd.HydrodynamicModel(
    hymd.FlowRegime(flow_regime),
    hymd.Porosity(porosity),
    hymd.Permeability(permeability, units="m/s"),
    hymd.DiffusionCoefficient(diffusion_coef, units="m2/s"),
    hymd.Head(0.0, "m"),
    hymd.AdvectionCrankNicolson(0.5),
    hymd.DiffusionCrankNicolson(1.0),
    hymd.StorageCoefficient(storage_coefficient),
)
simu_base.model.add_sub_item(hmodel)

- Geochemical unit definition

In [ ]:
chmodel = hymd.GeochemicalModel(hymd.Report("full"), hymd.Redox("disabled"))
# Define a geochem unit
base_unit = hymd.GeochemicalUnit("chem_base")
inj_unit = hymd.GeochemicalUnit("injected_solution")
tracer_species = "T_Cinet"
mineral_species = "Min_T_Cinet"
# Set concentrations
base_unit.add_sub_item(
    hymd.Concentration(tracer_species, c0, units="molal"),
)
base_unit.add_sub_item(
    hymd.Mineral(
        mineral_species,
        grade=1e-10,
        grade_units="mol/kg",
        surface=surface,
        surface_units="cm2/g",
    )
)
inj_unit.add_sub_item(hymd.Concentration(tracer_species, c_inj, units="molal"))
# define tracer
chmodel.add_sub_item(
    hymd.Define("basis", tracer_species, hymd.MoleWeight(moleweight, units="g/mol"))
)
chmodel.add_sub_item(
    hymd.Define(
        "mineral",
        mineral_species,
        hymd.Composition(f"1 {tracer_species}"),
        hymd.Surface(surface, units="cm2/g"),
        hymd.LogK(logK),
        hymd.Kinetics(
            hymd.Rate(kv, units="mol/m2/s"),
            hymd.Area(mineral_species),
            hymd.Yterm(hymd.Species(mineral_species)),
        ),
    )
)

# exclude
chmodel.add_sub_item(hymd.Exclude(("minerals", "colloids", "gases")))

# Add the units to the geochemical model
chmodel.add_sub_item(base_unit)
chmodel.add_sub_item(inj_unit)


# Add to the main model
simu_base.model.add_sub_item(chmodel)

- Geometry definition

In [ ]:
gmodel = hymd.GeometryModel(
    hymd.GridRegime("rectangle"),
    hymd.Domain(f"{nx*dx},{nx} {ny*dy},{ny}"),
    hymd.Zone("domain", hymd.Geometry("domain"), hymd.Geochemistry(base_unit.name)),
)
simu_base.model.add_sub_item(gmodel)

- Boundary model definition -> No boundary

In [ ]:
# flc_left = hymd.FlowCondition(f"constant-head at {cst_head_left} m")
# flc_right = hymd.FlowCondition(f"constant-head at {cst_head_right} m")
# bmodel = hymd.BoundaryModel(
#     hymd.Boundary("border_left", hymd.Coordinates(f"0,0, 0,{ny*dy}", units="m"), flc_left),
#     hymd.Boundary(
#         "border_right", hymd.Coordinates(f"{nx*dx},{ny*dy}, {nx*dx},0", units="m"), flc_right
#     ),
# )
# simu_base.model.add_sub_item(bmodel)

- Time discretization

In [ ]:
tmodel = hymd.TimeDiscretizationModel(
    hymd.Duration(nt * dt, units="s"),
    hymd.TimeStep(
        hymd.Variable(
            hymd.StartValue(dt, "s"), hymd.Maximum(dt, "s"), hymd.CourantFactor(20.0)
        ),
    ),
)
simu_base.model.add_sub_item(tmodel)

- Sampling definition

In [ ]:
# Sampling model
smodel = hymd.SamplingModel(
    hymd.GridSampling(min(30, nt)),
    hymd.Select("time", units="s"),
    hymd.Select("node-number"),
    hymd.Select("x-flowrate", units="m/s"),
    hymd.Select("y-flowrate", units="m/s"),
    hymd.Select("permeability", units="m/s"),
    hymd.Select("head", units="m"),
    hymd.Select("porosity"),
    hymd.Select("diffusion"),
    hymd.Select(tracer_species, units="mol/l"),
    hymd.Select(mineral_species, units="mol/kg"),
    hymd.FluxSampling(duration_in_days),
    hymd.FluxSelect(tracer_species, units="mol/s"),
)


simu_base.model.add_sub_item(smodel)

- Create the injection/pumping file: 1 column per well + 1 column for the time (days)

In [ ]:
src_term_path: str = "MODIFY/source_terms.dat"
src_term_data: NDArrayFloat = np.zeros(
    (duration_in_days + 1, len(inj_grid_locations) + len(prod_grid_locations) + 1)
)
src_term_data[:, 0] = time_in_days

- Create one zone per well with the correct flowrates: negative for producers and positive for injectors.

In [ ]:
src_term_data.shape

In [ ]:
prod_flw = -flowrates  # m3/h
times = np.arange(stop=nt + 1, start=0) * dt

# 1) Add the producer wells
count_prod: int = 0  # to avoid unbounded values in the next loop

for count_prod, (x, y) in enumerate(prod_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    name = f"producer_{ix}_{iy}"

    zone = hymd.Zone(name)
    zone.add_sub_item(
        hymd.Geometry(
            f"rectangle {(ix + 0.5) * dx},{(iy + 0.5) * dy}, {dx},{dy}",
            units="m",
        )
    )
    zone.add_sub_item(hymd.Geochemistry(base_unit.name))
    zone.add_sub_item(hymd.GlobalFlux(name))
    zone.add_sub_item(
        hymd.Source(prod_flw[0], units="m3/h", geochem_unit=inj_unit.name)
    )
    zone.add_sub_item(
        hymd.Modify(
            "$1",
            hymd.Source(
                f"${count_prod+2}",
                units="m3/h",
                geochem_unit=inj_unit.name,
                src_file_path=src_term_path,
            ),
            time_units="h",
        )
    )
    simu_base.model.get_sub_model(hymd.GeometryModel).add_sub_item(zone)
    # Update the src_terms file -> 2 m3/h for a producer
    src_term_data[:, count_prod + 1] = prod_flw

    base_model.add_src_term(sink_term)


# 2) Add the injector wells
for count_inj, (x, y) in enumerate(inj_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    name = f"injector_{ix}_{iy}"

    nb_prod_well_linked = 0
    # Find the number of producers that the injector is linked with
    for xp, yp in prod_grid_locations:
        # we take 20% margin on the cell radius
        if np.sqrt((x - xp) ** 2 + (y - yp) ** 2) < cell_radius * 1.2:
            nb_prod_well_linked += 1
    inj_flowrates = -prod_flw / 6.0 * nb_prod_well_linked

    zone = hymd.Zone(name)
    zone.add_sub_item(
        hymd.Geometry(
            f"rectangle {(ix + 0.5) * dx},{(iy + 0.5) * dy}, {dx},{dy}",
            units="m",
        )
    )
    zone.add_sub_item(hymd.Geochemistry(base_unit.name))
    zone.add_sub_item(hymd.GlobalFlux(name))
    zone.add_sub_item(
        hymd.Source(f"{inj_flowrates[0]}", units="m3/h", geochem_unit=inj_unit.name),
    )
    zone.add_sub_item(
        hymd.Modify(
            "$1",
            hymd.Source(
                f"${count_prod + count_inj +3}",
                units="m3/h",
                geochem_unit=inj_unit.name,
                src_file_path=src_term_path,
            ),
            time_units="h",
        )
    )
    simu_base.model.get_sub_model(hymd.GeometryModel).add_sub_item(zone)

    src_term_data[:, count_prod + count_inj + 2] = +inj_flowrates

# 3) Add the src_data_file (need to write and read the file... which is a bit stupid...)
# Register the source data file
# Note: need a high number of digits to get something strictly equivalent with the demonstrator
simu_base.register_modifiy_src_file(src_term_path, src_term_data, fmt="%.6f")

In [ ]:
# negative
total_prod_flow = (
    -np.sum(src_term_data[:, 1 : len(prod_grid_locations) + 1]) * dt / 3600
)
# positive
total_inj_flow = np.sum(src_term_data[:, len(prod_grid_locations) + 1 :]) * dt / 3600

print(f"total_prod_flow = {total_prod_flow} m3")
print(f"total_inj_flow = {total_inj_flow} m3")

# This test should fail if there is any issue !
assert np.round(total_prod_flow, 4) == np.round(total_inj_flow, 4)

- Define a runner

In [ ]:
if runner_type == RunnerType.FRONTAL:
    runner = FrontalHytecRunner(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        nb_cpu=4,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.SLURM:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=4,
        queue="geo-cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = SlurmHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.QSUB:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=4,
        queue="Omines_cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = QsubHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
else:
    raise Exception("Could not created runner")

- Create four models from this base simulation

In [ ]:
simu_reference = copy.deepcopy(simu_base)
simu_reference.update_root_and_name(
    new_root="simu_reference", new_name="simu_reference"
)
simu_estimate_flat = copy.deepcopy(simu_base)
simu_estimate_flat.update_root_and_name(
    new_root="simu_estimate_flat", new_name="simu_estimate"
)
simu_estimate_simu_1 = copy.deepcopy(simu_base)
simu_estimate_simu_1.update_root_and_name(
    new_root="simu_estimate_simu_1", new_name="simu_estimate_simu_1"
)
simu_estimate_simu_2 = copy.deepcopy(simu_base)
simu_estimate_simu_2.update_root_and_name(
    new_root="simu_estimate_simu_2", new_name="simu_estimate_simu_2"
)

- Add the permeability fields to the simulations

In [ ]:
index: NDArrayFloat = np.arange(nx * ny, dtype=np.int32)

# True
data_base = pd.DataFrame(
    data={
        "node-number": index,
        # x and y are cell centers
        # "x": (index % nx) * dx + dx / 2,
        mineral_species: 0.0,
    },  # need to flatten the parameter
    index=index,
)

data_reference = data_base.copy()
data_reference[mineral_species] = reference_grade.flatten("F") * conv_u_grade
simu_reference.add_param_file_data(ParameterFiles.MINERALS, data_reference)

data_estimate_flat = data_base.copy()
data_estimate_flat[mineral_species] = estimate_grade_flat.flatten("F") * conv_u_grade
simu_estimate_flat.add_param_file_data(ParameterFiles.MINERALS, data_estimate_flat)

data_estimate_simu_1 = data_base.copy()
data_estimate_simu_1[mineral_species] = estimate_grade_simu.flatten("F") * conv_u_grade
simu_estimate_simu_1.add_param_file_data(ParameterFiles.MINERALS, data_estimate_simu_1)

data_estimate_simu_2 = data_base.copy()
data_estimate_simu_2[mineral_species] = estimate_grade_simu2.flatten("F") * conv_u_grade
simu_estimate_simu_2.add_param_file_data(ParameterFiles.MINERALS, data_estimate_simu_2)

- Define a runner

In [ ]:
if runner_type == RunnerType.FRONTAL:
    runner = FrontalHytecRunner(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        nb_cpu=4,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.SLURM:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=12,
        queue="geo-cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = SlurmHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.QSUB:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=12,
        queue="Omines_cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = QsubHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
else:
    raise Exception("Could not created runner")

- Write input files

In [ ]:
for simu in [
    simu_reference,
    simu_estimate_flat,
    simu_estimate_simu_1,
    simu_estimate_simu_2,
]:
    simu.write_input_files()

- Run simulations

In [ ]:
for simu in [
    simu_reference,
    simu_estimate_flat,
    simu_estimate_simu_1,
    simu_estimate_simu_2,
]:
    runner.run(simu)

- Reading the results and displaying available fields

In [ ]:
for simu in [
    simu_reference,
    simu_estimate_flat,
    simu_estimate_simu_1,
    simu_estimate_simu_2,
]:
    simu.read_hytec_results()

In [ ]:
simu_reference.handlers.results.grid_res_columns

In [ ]:
simu_reference.handlers.results.flux_res_columns

In [ ]:
# Get the coordinates so we can plot the velocities
x_grid = simu_reference.handlers.results.extract_field_from_grid_res(
    field="x-distance", nx=nx, ny=ny
)
y_grid = simu_reference.handlers.results.extract_field_from_grid_res(
    field="y-distance", nx=nx, ny=ny
)

# Get heads and velocities
x_velocities = simu_reference.handlers.results.extract_field_from_grid_res(
    field="x-flowrate [m/s]", nx=nx, ny=ny
)
y_velocities = simu_reference.handlers.results.extract_field_from_grid_res(
    field="y-flowrate [m/s]", nx=nx, ny=ny
)
fwd_heads_grid_true_hytec = simu_reference.handlers.results.extract_field_from_grid_res(
    field="head [m]", nx=nx, ny=ny
)

# Getting sample time. The unit is the same than the one defined for the simulation duration in the htc
grid_sample_times = simu_reference.handlers.results.get_sample_times_from_grid_res()
grid_sample_times_days = grid_sample_times / 3600 / 24
hytec_flux_sample_times = simu_reference.handlers.results.flux_res_data[0, :, 0]

In [ ]:
hytec_fwd_conc_grid = {}
hytec_fwd_conc_flux = {}
hytec_grade = {}


for simu in [
    simu_reference,
    simu_estimate_flat,
    simu_estimate_simu_1,
    simu_estimate_simu_2,
]:
    simu.read_hytec_results()

    hytec_fwd_conc_grid[simu.name] = (
        simu.handlers.results.extract_field_from_grid_res(
            field="T_Cinet [mol/l]", nx=nx, ny=ny
        )[:, :, 0, :]
        / conv_u_conc
    )
    hytec_grade[simu.name] = (
        simu.handlers.results.extract_field_from_grid_res(
            field="Min_T_Cinet [mol/kg]", nx=nx, ny=ny
        )[:, :, 0, :]
        / conv_u_grade
    )
    # Getting flux results for the column "T_Cinet [mol/s]" (index 1)
    hytec_fwd_conc_flux[simu.name] = simu.handlers.results.extract_field_from_flux_res(
        "T_Cinet [mol/s]"
    )

- Check if the grade field has been correctly updated in the HYTEC simulations

In [ ]:
plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (14, 3.5)},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "axes1-4"]], sharey=True, sharex=True
        )
    },
)

ngp.multi_imshow(
    axes=plotter.axes,
    fig=plotter.fig,
    data={
        "Reference": hytec_grade[simu_reference.name][:, :, 0],
        "Cst estimate": hytec_grade[simu_estimate_flat.name][:, :, 0],
        "Cond sim dat ok": hytec_grade[simu_estimate_simu_1.name][:, :, 0],
        "Cond sim dat bad": hytec_grade[simu_estimate_simu_2.name][:, :, 0],
    },
    cbar_title="Field value",
    imshow_kwargs={
        "cmap": plt.get_cmap("jet"),
        "extent": [0.0, nx * dx, 0.0, ny * dy],
        "vmin": 50,
        "vmax": 650,
    },
    xlabel="X (m)",
    ylabel="Y (m)",
)

- Plot the charges and the velocity

In [ ]:
time_index = 1

plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (11, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)

ngp.multi_imshow(
    axes=plotter.axes,
    fig=plotter.fig,
    data={
        "PyRTID": model_reference.fl_model.head[:, :, time_index * 4],
        "HYTEC": fwd_heads_grid_true_hytec[:, :, 0, time_index],
    },
    cbar_title="Head $[m]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy], "aspect": "equal"},
)

Y, X = np.meshgrid((np.arange(nx) + 0.5) * dx, (np.arange(ny) + 0.5) * dy)
plotter.ax_dict["ax1-1"].quiver(
    X,
    Y,
    model_reference.fl_model.u_darcy_x_center[:, :, time_index * 4],
    model_reference.fl_model.u_darcy_y_center[:, :, time_index * 4],
    color="C0",
    scale_units="xy",
)

plotter.ax_dict["ax1-2"].quiver(
    X,
    Y,
    x_velocities[:, :, 0, time_index],
    y_velocities[:, :, 0, time_index],
    color="C0",
    scale_units="xy",
)

# plotter.ax_dict["ax1-1"].set_title("Heads [m]", fontweight="bold")
# plotter.ax_dict["ax1-2"].set_title("Darcy velocity [m/s]", fontweight="bold")

plotter.subfigs["fig0"].suptitle(
    "Heads and Darcy velocities", fontweight="bold", fontsize=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

fname = "heads_and_velocities"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Plot the concentration evolution

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (16, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "ax1-4"]], sharey=True, sharex=True
        ),
    },
)
nb_frames = min(15, hytec_fwd_conc_grid[simu_reference.name].shape[-1])

plotter.animated_multi_imshow(
    ax_names=list(plotter.ax_dict.keys()),
    fig=plotter.fig,
    data={
        "Reference": hytec_fwd_conc_grid[simu_reference.name],
        "Estimate (flat)": hytec_fwd_conc_grid[simu_estimate_flat.name],
        "Estimate (simu_1)": hytec_fwd_conc_grid[simu_estimate_simu_1.name],
        "Estimate (simu_2)": hytec_fwd_conc_grid[simu_estimate_simu_2.name],
    },
    cbar_title="Concentration $[mg/l]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy], "aspect": "equal"},
    nb_frames=nb_frames,
    # is_add_grid=True,
)

# Add animated text
plotter.plot_animated_text(
    plotter.ax_dict["ax1-1"],
    x=5.0,
    y=290.0,
    s=[f"{d:.2f} d" for d in np.arange(nb_frames) * duration_in_days / (nb_frames - 1)],
    fontweight="bold",
)

plotter.subfigs["fig0"].suptitle(
    "Concentration evolution (PyRTID)", fontweight="bold", size=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("fwd_conc_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(str(fname_html), writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

In [ ]:
# Plot a grade evolution animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (16, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(
            mosaic=[["ax1-1", "ax1-2", "ax1-3", "ax1-4"]], sharey=True, sharex=True
        ),
    },
)
nb_frames = nb_frames = min(15, model_reference.time_params.nt)

plotter.animated_multi_imshow(
    ax_names=list(plotter.ax_dict.keys()),
    fig=plotter.fig,
    data={
        "Reference": hytec_grade[simu_reference.name],
        "Estimate (flat)": hytec_grade[simu_estimate_flat.name],
        "Estimate (simu_1)": hytec_grade[simu_estimate_simu_1.name],
        "Estimate (simu_2)": hytec_grade[simu_estimate_simu_2.name],
    },
    cbar_title="Concentration $[mg/l]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy], "aspect": "equal"},
    nb_frames=nb_frames,
    # is_add_grid=True,
)

# Add animated text
plotter.plot_animated_text(
    plotter.ax_dict["ax1-1"],
    x=5.0,
    y=290.0,
    s=[f"{d:.2f} d" for d in np.arange(nb_frames) * duration_in_days / (nb_frames - 1)],
    fontweight="bold",
)

plotter.subfigs["fig0"].suptitle("Grade evolution (PyRTID)", fontweight="bold", size=16)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("fwd_conc_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(str(fname_html), writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Plot the concentrations at the production wells

In [ ]:
# Indicate the order in which to plot
obs_plot_locations = ["ax2-3", "ax1-2", "ax3-2", "ax2-2", "ax1-1", "ax3-1", "ax2-1"]

plotter = ngp.NestedGridPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 8)},
    subfigs_params={
        "ncols": 3  # np.unique(np.array(prod_grid_locations)[:, 0].round()).size,
    },
    subplots_mosaic_params={
        "left_col": dict(
            mosaic=[["ghost1"], ["ax1-1"], ["ax1-2"], ["ghost2"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
        "centered_col": dict(
            mosaic=[["ax2-1"], ["ax2-2"], ["ax2-3"]], sharey=True, sharex=True
        ),
        "right_col": dict(
            mosaic=[["ghost3"], ["ax3-1"], ["ax3-2"], ["ghost4"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
    },
)

# hide axes and borders
plotter.ax_dict["ghost1"].axis("off")
plotter.ax_dict["ghost2"].axis("off")
plotter.ax_dict["ghost3"].axis("off")
plotter.ax_dict["ghost4"].axis("off")

# Static plot
for count, ((x, y), ax_name) in enumerate(zip(prod_grid_locations, obs_plot_locations)):
    ix = int(x // dx)
    iy = int(y // dy)
    name = f"producer_{ix}_{iy}"
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ x={x:.1f}m, y={y:.1f}m"

    ax = plotter.ax_dict[ax_name]
    ax.set_title(obs_well_name, fontweight="bold", fontsize=12)

    well_name = f"producer_{ix}_{iy}"

    data = {
        "Reference.": {
            "data": -simu_reference.handlers.results.extract_field_from_flux_res(
                "T_Cinet [mol/s]", well_name
            ),
            "kwargs": {"c": "b"},
        },
        "A priori cst.": {
            "data": -simu_estimate_flat.handlers.results.extract_field_from_flux_res(
                "T_Cinet [mol/s]", well_name
            ),
            "kwargs": {"c": "r"},
        },
        "A priori simu_1.": {
            "data": -simu_estimate_simu_1.handlers.results.extract_field_from_flux_res(
                "T_Cinet [mol/s]", well_name
            ),
            "kwargs": {"c": "g", "linestyle": "--"},
        },
        "A priori simu_2": {
            "data": -simu_estimate_simu_2.handlers.results.extract_field_from_flux_res(
                "T_Cinet [mol/s]", well_name
            ),
            "kwargs": {"c": "k", "linestyle": "--"},
        },
    }

    for k, v in data.items():
        ax.plot(v["data"], **v["kwargs"], label=k)

    ax.set_xlabel("Time", fontweight="bold")
    ax.set_xlabel("T_Cinet [mol/s]", fontweight="bold")

plotter.add_fig_legend(ncol=2)
plotter.fig.supxlabel("Time [d]", fontweight="bold")
plotter.fig.supylabel("T_Cinet [mol/s]", fontweight="bold")

fname = "T_Cinet_true_at_producers"
for format in ["png", "pdf"]:
    plotter.savefig(str(fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Animation of the grade mineral grade evolution

- Plot the concentrations at the producer wells. Note, we add some white noise to the "True" data :)

In [ ]:
rng = np.random.default_rng(2021)
noise_std = 5e-5  # This is an absolute value


def make_noisy(x: NDArrayFloat) -> NDArrayFloat:
    """Return the input with some added white noise.

    Note
    ----
    The parameters are hardcoded to be consistent in the notebook.
    Change the function directly.
    """
    mean_noise = 0.0  # mean
    return x + rng.normal(mean_noise, noise_std, x.shape)

In [ ]:
noise_std * conv_u_conc

In [ ]:
# Indicate the order in which to plot
obs_plot_locations = ["ax2-3", "ax1-2", "ax3-2", "ax2-2", "ax1-1", "ax3-1", "ax2-1"]

plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 8)},
    subfigs_params={
        "ncols": 3  # np.unique(np.array(prod_grid_locations)[:, 0].round()).size,
    },
    subplots_mosaic_params={
        "left_col": dict(
            mosaic=[["ghost1"], ["ax1-1"], ["ax1-2"], ["ghost2"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
        "centered_col": dict(
            mosaic=[["ax2-1"], ["ax2-2"], ["ax2-3"]], sharey=True, sharex=True
        ),
        "right_col": dict(
            mosaic=[["ghost3"], ["ax3-1"], ["ax3-2"], ["ghost4"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
    },
)

# hide axes and borders
plotter.ax_dict["ghost1"].axis("off")
plotter.ax_dict["ghost2"].axis("off")
plotter.ax_dict["ghost3"].axis("off")
plotter.ax_dict["ghost4"].axis("off")

# Static plot
count = 0  # to avoid unbounded linting
for count, ((x, y), ax_name) in enumerate(zip(prod_grid_locations, obs_plot_locations)):
    ix = int(x // dx)
    iy = int(y // dy)
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ \n x={x:.1f}m, y={y:.1f}m"

    plotter.plot_1d_static(
        ax_name=ax_name,
        title=obs_well_name,
        data={
            'Reference ("True") + white noise': {
                # Note: we add some white noise
                "data": (
                    grid_sample_times_days,
                    np.abs(
                        make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :]) * conv_u_conc
                    ),
                ),
                "kwargs": {"marker": "o", "linestyle": "None", "c": "b", "alpha": 0.5},
            },
            "Initial estimation": {
                "data": (
                    grid_sample_times_days,
                    np.abs(fwd_conc_grid_estimate_hytec[ix, iy, 0, :]) * conv_u_conc,
                ),
                "kwargs": {"c": "r"},
            },
        },
        xlabel="Time [d]",
        ylabel="UO2[2+] [mg/l]",
    )

ymax: float = (
    np.nanmax(
        np.abs(
            make_noisy(fwd_conc_grid_true_hytec),
            np.abs(fwd_conc_grid_estimate_hytec),
        )
    )
) * conv_u_conc
for ax in plotter.ax_dict.values():
    ax.set_ylim(-0.1 * ymax, ymax)

plotter.add_fig_legend(ncol=2)
# plotter.fig.supxlabel("Time [d]", fontweight="bold")
# plotter.fig.supylabel("UO2[2+] [mol/l]", fontweight="bold")

plotter_wells = plotter

fname = "T_Cinet_true_at_producers"
for format in ["png", "pdf"]:
    plotter.savefig(str(fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Check the objective function: with flux results

In [ ]:
fun = 0
for count, (x, y) in enumerate(prod_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    fun += np.sum(
        np.square(
            np.abs(make_noisy(fwd_conc_flux_true_hytec[1:, count]))
            - np.abs(fwd_conc_flux_estimate_hytec[1:, count])
        )
    )
fun = 0.5 * fun / noise_std / noise_std

print("J0 - flux = ", fun)
print("vmult = ", 1 / fun)

- With grid results

In [ ]:
fun = 0
for count, (x, y) in enumerate(prod_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    fun += np.sum(
        np.square(
            np.abs(make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :]))
            - np.abs(fwd_conc_grid_estimate_hytec[ix, iy, 0, :])
        )
    )
fun = 0.5 * fun / noise_std / noise_std

print("J0 = ", fun)
print("vmult = ", 1 / fun)

## Inversion - first gradient check

- Creation of a base simulation for inversion with observation files, and the associated zones in the htc file.

In [ ]:
simu_inverse = copy.deepcopy(simu_estimate)
simu_inverse.update_root_and_name(new_root="simu_inverse", new_name="simu_inverse")

simu_inverse.handlers.htc.samples = nsamples

# 1) Add the observation wells
for count, (x, y) in enumerate(prod_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)
    name = f"producer_{ix}_{iy}"

    # Update the zone from the htc file
    zone = simu_inverse.handlers.htc.zones_dict[name]

    # Add the observables for the area
    obs = Observable(
        zone_name=zone.name,
        column_index=3,
        root_relative_path=f"observables/well{count + 1}_o.dat",
        root_path=simu_inverse.root,
        type=ObservationType.GRID,
        uncertainties_column_index=4,
    )

    # vals = np.abs(make_noisy(fwd_conc_flux_true_hytec[:, count]))  # must be positive for now
    vals = np.abs(make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :]))
    odata = pd.DataFrame(
        data={
            "time [s]": grid_sample_times,  # flux_sample_times,  # 2022/09/08 -> no unit support yet, should be in seconds
            "node-number": indices_to_hytec_node_number(ix, nx=nx, iy=iy, ny=ny, iz=0),
            "T_Cinet [mol/l]": vals,
            "T_Cinet-uncertainty [mol/l]": noise_std,
        }
    )
    obs.handler.update_data(odata)
    zone.observables.append(obs)

# 2) Update the htc file with some options
simu_inverse.handlers.htc.optimization = "enabled"
simu_inverse.handlers.htc.optimization_solver = OptimizationSolverConfig(
    solver_name="lbfgsb",
    max_number_iterations=1,
    max_number_fwd_model_eval=10,
    max_number_gradient_eval=1,
    max_number_hessian_eval=1,
    objfun_threshold=1e-1,
    objfun_min_change=0.0,
    param_min_change=0.0,
    gradient_min_norm=0.0,
    hessian_max_norm=0.0,
    grad_history_size=5,
    adjoint_state="enabled",
    fd_gradient_check="disabled",
)
simu_inverse.handlers.htc.adjusted_parameters_dict = {
    "grade": AdjustedParameterConfig(
        name="mineral grade",
        lbounds=0.0,
        ubounds=700 * conv_u_grade,  # 700 ppm
    )
}
simu_inverse.handlers.htc.adjoint_sampling = (
    nsamples  # number of samples on the adjoint variables
)

# 3)Write the input files
simu_inverse.write_input_files()

# Run !
runner.run(simu_inverse)

simu_inverse.read_hytec_results()

print("objective function J0 = ", simu_inverse.handlers.results.optim_res.obj_funs[0])
print("vmult = ", 1 / simu_inverse.handlers.results.optim_res.obj_funs[0])

- Extract the adjoint variables computed for the first gradient

In [ ]:
print(f"columns = {simu_inverse.handlers.results.optim_res.adjoint_var_columns}")

In [ ]:
adj_conc_hytec = simu_inverse.handlers.results.extract_field_from_adj_var_res(
    field="adjoint-variable{T_Cinet} [m]", nx=nx, ny=ny
)
adj_min_hytec = simu_inverse.handlers.results.extract_field_from_adj_var_res(
    field="adjoint-variable{T_Cinet} [m]", nx=nx, ny=ny
)
adj_conc_hytec.shape

- Plot the adjoint variable evolution

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (10, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)
nb_frames = nsamples

plotter.plot_2d_animated(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "Adj T_Cinet": adj_conc_hytec[:, :, 0, ::-1],
        "Adj_grade": adj_min_hytec[:, :, 0, ::-1],
    },
    cbar_title="Concentration $[molal]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    nb_frames=nb_frames,
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle(
    "Adjoint variables (conc and min)", fontweight="bold", size=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("adj_variables_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Extract and plot the gradient

In [ ]:
# Get the HYTEC  Adj gradient
hytec_fd_gradients = simu_inverse.handlers.results.optim_res.fd_gradients
hytec_adjoint_gradient = simu_inverse.handlers.results.optim_res.adjoint_gradients

In [ ]:
grad_values: NDArrayFloat = (
    hytec_adjoint_gradient[0].loc[:, "value"].to_numpy().reshape(ny, nx)
)

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (10, 5)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)

plotter.plot_2d_static(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "Grad": grad_values,
        "Grad2": grad_values,
    },
    cbar_title="Concentration $[molal]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle(
    "Adjoint variables (conc and min)", fontweight="bold", size=16
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

## Inversion - full optimization

In [ ]:
simu_inverse_full = copy.deepcopy(simu_inverse)
simu_inverse_full.update_root_and_name(
    new_root="simu_inverse_full", new_name="simu_inverse_full"
)

# 2) Update the htc file with some options
simu_inverse_full.handlers.htc.optimization_solver = OptimizationSolverConfig(
    solver_name="lbfgsb",
    max_number_iterations=12,
    max_number_fwd_model_eval=12,
    max_number_gradient_eval=12,
    max_number_hessian_eval=1,
    objfun_threshold=1e-1,
    objfun_min_change=0.0,
    param_min_change=0.0,
    gradient_min_norm=0.0,
    hessian_max_norm=0.0,
    grad_history_size=5,
    adjoint_state="enabled",
    fd_gradient_check="disabled",
)

# 3)Write the input files
simu_inverse_full.write_input_files()

# Run !
runner.run(simu_inverse_full)

simu_inverse_full.read_hytec_results()

# print("objective function J0 = ", simu_inverse_full.handlers.results.optim_res.obj_funs)
# print("vmult = ", 1/ simu_inverse.handlers.results.optim_res.obj_funs[0])

- Get the concentrations

In [ ]:
# Get the results on a 3D grid with the last dimension as time step
fwd_conc_inversed_hytec = (
    simu_inverse_full.handlers.results.extract_field_from_grid_res(
        field="T_Cinet [mol/l]", nx=nx, ny=ny
    )
)

- Get the gradients

In [ ]:
simu_inverse_full.handlers.results.optim_res.adjoint_gradients[0].columns

adjoint_gradients = simu_inverse_full.handlers.results.get_adjoint_gradient(
    "grade", nx=nx, ny=ny
)
fd_gradients = simu_inverse_full.handlers.results.get_fd_gradient("grade", nx=nx, ny=ny)
adjusted_mineral = simu_inverse_full.handlers.results.get_adjusted_parameter_field(
    "grade", nx=nx, ny=ny
)
obj_funs = simu_inverse_full.handlers.results.optim_res.obj_funs

In [ ]:
adjusted_mineral.shape

In [ ]:
adjoint_gradients.shape

In [ ]:
obj_funs.shape

- Align gradients and objective functions: Because of the construction in HYTEC, unless I am mistaken, the objective function and fitted parameter vectors match exactly. On the other hand, there is an uncontrolled mismatch with the gradients due to the solver which, depending on its needs, may have to apply several objective functions in a row or several gradients. This function makes sure that the two match for a good rendering of the animations.

In [ ]:
grad_index = 0
grad_obj_funs = (
    simu_inverse_full.handlers.results.optim_res.obj_funs_matching_adjoint_gradients
)

# Example: [3, 4, 4, 5, 7, 8, 8, 9]

aligned_adjoint_gradients = np.empty(
    (*adjoint_gradients[:, :, :, 0].shape, obj_funs.size)
)
aligned_adjoint_gradients.fill(np.nan)

previous_index: int = -1
for i, index in enumerate(grad_obj_funs):
    if index == previous_index:
        continue
    if index > obj_funs.size:
        continue
    previous_index = index
    aligned_adjoint_gradients[:, :, :, index - 1] = adjoint_gradients[:, :, :, i]

# Fill the first missing gradients
# Gradients 1 and 2 in the example
for i2 in range(grad_obj_funs[0] - 1):
    aligned_adjoint_gradients[:, :, :, i2] = aligned_adjoint_gradients[
        :, :, :, grad_obj_funs[0]
    ]

In [ ]:
aligned_adjoint_gradients.shape

- Get the observations and the predictions vector to plot it against each others

In [ ]:
obs_vector = simu_inverse_full.get_observation_vector()
pred_vector = simu_inverse_full.get_results_matching_obs_vector(
    simu_inverse_full.get_observables()
)

In [ ]:
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (15, 9)},
    subfigs_params={"nrows": 2, "ncols": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2", "ax1-3"]], sharex=True, sharey=True),
        "fig1": dict(mosaic=[["ax2-1", "ax2-2"]]),
    },
)

# 1 frame per solver iteration
nb_frames: int = obj_funs.shape[0]

# 1) Gradient evolution
plotter.plot_2d_animated(
    ax_names=["ax1-1"],
    data={
        "Gradient": aligned_adjoint_gradients[:, :, 0, :],
    },
    # title="Gradient",
    xlabel="Node x#",
    ylabel="Node y#",
    nb_frames=nb_frames,
    is_symmetric_cbar=True,
)

# 2) Parameter evolution
plotter.plot_2d_animated(
    ax_names=["ax1-2", "ax1-3"],
    data={
        "Uraninite inverted": adjusted_mineral[:, :, 0, :] / conv_u_grade,
        "Uraninite true": np.transpose(
            np.repeat(true_grade[None, :, :], nb_frames, axis=0), axes=(1, 2, 0)
        )
        / conv_u_grade,
    },
    # title="Gradient",
    xlabel="Node x#",
    ylabel="Node y#",
    nb_frames=nb_frames,
    cbar_title="Uraninite $[ppm]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet"), "vmin": 100, "vmax": 650},
)

# 3) Objective function
vals = obj_funs
obj_fun_vals = np.full((nb_frames, len(vals)), fill_value=np.nan)
for i in range(len(vals)):
    obj_fun_vals[i, : i + 1] = vals[: i + 1]

plotter.animated_multi_plot(
    ax_name="ax2-1",
    nb_frames=nb_frames,
    data={
        "Obj fun": {"data": obj_fun_vals.T, "kwargs": {"c": "r", "linestyle": "--"}},
    },
    title="Objective function",
    xlabel="Iteration #",
)
plotter.ax_dict["ax2-1"].set_yscale("log")

# NOTE: 2022/09/07 -> need to find a way to make that less heavy
# 4) Observation vs predicted values

plot_observed_vs_simulated(
    plotter.ax_dict["ax2-2"],
    obs_vector=obs_vector * conv_u_conc,
    pred_vector=pred_vector * conv_u_conc,
    pred_vector_initial=simu_estimate.get_results_matching_obs_vector(
        simu_inverse_full.get_observables()
    )
    * conv_u_conc,
    unit="$mg/l^{-1}$",
)

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("m_j_g_animation.html")
writer = HTMLWriter(fps=1, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Check what it gives at the cell scale

In [ ]:
# Compute the reserve associated to the cells in t
# Need to transpose for the display
inverted_reserves = get_cell_reserves(
    adjusted_mineral[:, :, 0, -1].T.ravel(), polygons, dx, dy, dz
)

plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (11, 4)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2", "ax1-3"]], sharey=True, sharex=True)
    },
)

for ax, reserves in zip(
    plotter.ax_dict.values(), (estimated_reserves, true_reserves, inverted_reserves)
):
    for i, j in inj_grid_locations:
        ax.plot(i, j, "ko")

    for i, j in prod_grid_locations:
        ax.plot(i, j, "ro")

    patches = []
    for i, cell_polygon in enumerate(polygons):
        # Add the polygon to the collection of patches
        xy = np.array(cell_polygon)
        patches.append(Polygon(xy, closed=True, facecolor=None))

        centroid = (xy.mean(axis=0)[0], xy.mean(axis=0)[1] * 1.05)
        ax.text(*centroid, f"{reserves[i]:.2f} t", fontsize=10, fontweight="bold")

        # Plot the number of the polygon
    p = PatchCollection(patches, alpha=0.75, cmap=plt.get_cmap("jet"))
    # p.set_facecolors("white")
    p.set_edgecolors("black")
    p.set_linewidth(0.6)
    p.set_linestyle("-")
    p.set_array(np.array(reserves, dtype=float))
    # p.set_label(np.array(np.round(reserves, 2), dtype=float))
    p.set_clim(1, 5.5)

    ax.add_collection(p)


# Add a common colorbar
plotter.fig.colorbar(p, label="Uranium [t]")

# Box dimensions
for ax in plotter.ax_dict.values():
    ax.set_xlim(0, nx * dx)
    ax.set_ylim(0, ny * dy)
    ax.set_aspect("equal", adjustable="box")

plotter.subfigs["fig0"].suptitle(
    "Metal per production cell [t]", fontweight="bold", fontsize=18
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.ax_dict["ax1-1"].set_title("Initial guess", fontweight="bold")
plotter.ax_dict["ax1-2"].set_title('Reference ("True")', fontweight="bold")
plotter.ax_dict["ax1-3"].set_title("inverted", fontweight="bold")

fname = "reserves_per_cells_inversed"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

In [ ]:
inversed_u_dev = np.sum(inverted_reserves) - np.sum(true_reserves)
inversed_u_dev_frac = inversed_u_dev / np.sum(true_reserves)
print(f"initial_U_dev = {inversed_u_dev:.2f} t ({inversed_u_dev_frac * 100:.2f}%)")

- Export the cell tonnages and others to text files

In [ ]:
# export the reserves
np.savetxt("adjoint_inversed_reserves.txt", inverted_reserves)
# export the inverted mineral (in mol/kg)
np.savetxt("inverse_uraninite_field.txt", adjusted_mineral[:, :, 0, -1])

# Grid sample times
np.savetxt("grid_sample_times_days.txt", grid_sample_times_days)

# export the flux (in mg/l)
for count, ((x, y), ax_name) in enumerate(zip(prod_grid_locations, obs_plot_locations)):
    ix = int(x // dx)
    iy = int(y // dy)
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ \n x={x:.1f}m, y={y:.1f}m"

    # Observations
    np.savetxt(
        f"fwd_conc_ref_well{count}.txt",
        np.abs(make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :]) * conv_u_conc),
    )

    # Simulation (init)
    np.savetxt(
        f"fwd_conc_init_well{count}.txt",
        np.abs(fwd_conc_grid_estimate_hytec[ix, iy, 0, :]) * conv_u_conc,
    )

    np.savetxt(
        f"fwd_conc_inv_adj_well{count}.txt",
        np.abs(fwd_conc_inversed_hytec[ix, iy, 0, :]) * conv_u_conc,
    )

- Plot the concentrations at producer wells

In [ ]:
plotter = plotter_wells

# Static plot
count = 0  # to avoid unbounded linting
for count, ((x, y), ax_name) in enumerate(zip(prod_grid_locations, obs_plot_locations)):
    ix = int(x // dx)
    iy = int(y // dy)
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ \n x={x:.1f}m, y={y:.1f}m"

    plotter.plot_1d_static(
        ax_name=ax_name,
        data={
            "Post inversion": {
                "data": (
                    grid_sample_times_days,
                    np.abs(fwd_conc_inversed_hytec[ix, iy, 0, :]) * conv_u_conc,
                ),
                "kwargs": {"c": "green"},
            },
        },
    )


plotter.add_fig_legend(ncol=3)

fname = "T_Cinet_true_at_producers_post_inversion"
for format in ["png", "pdf"]:
    plotter.savefig(str(fig_save_path.joinpath(f"{fname}.{format}")), format=format)

plotter.fig